## Librerie

In [1]:
import os
import logging
import psutil
import chromadb
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

##  Configurazione Logging e Controllo Memoria

In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def check_memory():
    available_memory = psutil.virtual_memory().available / (1024 ** 3)
    logger.info(f"Memoria disponibile: {available_memory:.1f} GB")
    if available_memory < 4.0:
        logger.warning("Bassa memoria! Potrebbero esserci rallentamenti.")

check_memory()


INFO:__main__:Memoria disponibile: 3.4 GB


## Caricamento e Preprocessing dei Documenti

In [4]:
def load_documents(docs_folder="documents"):
    all_documents = []
    
    for file in os.listdir(docs_folder):
        file_path = os.path.join(docs_folder, file)

        if file.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        elif file.endswith(".txt"):
            loader = TextLoader(file_path)
        else:
            continue  # Ignora file non supportati

        documents = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        chunks = text_splitter.split_documents(documents)

        # Aggiunta metadati con nome file e tipo di dato
        for chunk in chunks:
            chunk.metadata["file"] = file
            chunk.metadata["type"] = "text"

        all_documents.extend(chunks)

    logger.info(f"Caricati {len(all_documents)} chunk da {len(os.listdir(docs_folder))} file.")
    return all_documents

documents = load_documents()


INFO:__main__:Caricati 1674 chunk da 4 file.


## Creazione del Database Vettoriale (ChromaDB)

In [5]:
def create_vectorstore(documents, db_path="chromadb_index"):
    chroma_client = chromadb.PersistentClient(path=db_path)
    collection = chroma_client.get_or_create_collection(name="rag_collection")

    for i, doc in enumerate(documents):
        collection.add(
            documents=[doc.page_content], 
            metadatas=[doc.metadata], 
            ids=[str(i)]
        )

    logger.info("Database ChromaDB creato con successo.")
    return chroma_client, collection

chroma_client, collection = create_vectorstore(documents)


INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:__main__:Database ChromaDB creato con successo.


## Esportazione dei Chunk in un File di Testo

In [11]:
def export_chunks():
    chroma_client = chromadb.PersistentClient(path=db_path)
    collection = chroma_client.get_collection(name="rag_collection")
    results = collection.get(include=["documents", "metadatas"])

    export_path = os.path.join(db_path, "db_export.txt")
    
    with open(export_path, "w", encoding="utf-8") as file:
        for doc, meta in zip(results["documents"], results["metadatas"]):
            file.write(f"FILE: {meta['file']}\n")
            file.write(f"TIPO: {meta['type']}\n")
            file.write(f"CONTENUTO:\n{doc}\n")
            file.write("=" * 50 + "\n")

    logger.info(f"Esportazione completata: {export_path}")

export_chunks()

INFO:__main__:Esportazione completata: chromadb_index\db_export.txt


## Query del Database e Generazione Risposte

In [29]:
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

model_name = "llama3.2"
llm = ChatOllama(model=model_name)

## Funzione per Recupero Dati dal Database

In [30]:
def query_chromadb(query_text, n_results=3, db_path="chromadb_index"):
    chroma_client = chromadb.PersistentClient(path=db_path)
    collection = chroma_client.get_collection(name="rag_collection")
    results = collection.query(query_texts=[query_text], n_results=n_results)
    
    # Appiana eventuali liste annidate
    retrieved_docs = [doc for sublist in results["documents"] for doc in sublist]
    return retrieved_docs


## Creazione del Prompt e Generazione Risposta

In [31]:
template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question:
If you don't know the answer, then do not answer from your own knowledge.
Keep the answer concise.

#### Retrieved Context ####
{context}

#### Question ####
{question}

#### LLM Response ####
"""

prompt = ChatPromptTemplate.from_template(template)

def ask_question(question):
    retrieved_docs = query_chromadb(question)

    if not retrieved_docs:
        return "Nessuna informazione trovata nel database."

    # Unisce i documenti in un'unica stringa
    context = "\n".join(retrieved_docs)

    # Genera la risposta con il modello AI
    response = prompt.invoke({"context": context, "question": question})
    response = llm.invoke(response)
    response = StrOutputParser().invoke(response)

    return response


## Interfaccia da Terminale per Interrogare il Modello

In [ ]:
if __name__ == "__main__":
    while True:
        query = input("Inserisci una domanda (o 'exit' per uscire): ")
        if query.lower() == "exit":
            break
        print(ask_question(query))


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


I don't have a question to answer. You can ask me anything, and I'll do my best to help!


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


So, you want to know about special computer chips called GPUs.

GPU stands for Graphics Processing Unit. It's like a superhero for computers that helps them play games and show pretty pictures on the screen.

Imagine your computer is like a big brain, and it needs help with some of its tasks. That's where the GPU comes in! It's super good at doing lots of little math problems at the same time, which makes games and videos look really cool.

There are different kinds of GPUs, but some of the biggest companies that make them are Nvidia, AMD, and Intel. They're always trying to make their GPUs better and faster so they can help computers do more amazing things!

That's it!


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


It seems you've forgot to ask a question! Could you please provide a question for me to answer? I'll do my best to assist you.
